Original

https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/udacity/4_convolutions.ipynb

Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
train_dataset = save['train_dataset']
train_labels = save['train_labels']
valid_dataset = save['valid_dataset']
valid_labels = save['valid_labels']
test_dataset = save['test_dataset']
test_labels = save['test_labels']
del save  # hint to help gc free up memory
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
            / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
def convnet_5(image_size,
              num_channels,
              num_labels,
              patch_size,
              depth,
              hidden_size):
    X = tf.placeholder(shape=(None, image_size, image_size, num_channels), dtype=tf.float32)
    y = tf.placeholder(shape=(None, num_labels), dtype=tf.float32)

    # Layer 1
    layer1_weights = tf.Variable(tf.truncated_normal(
        shape=(patch_size, patch_size, num_channels, depth), stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros(shape=(depth,)))
    
    conv1 = tf.nn.conv2d(X, layer1_weights, strides=[1, 2, 2, 1], padding='SAME')
    hidden1 = tf.nn.relu(conv1 + layer1_biases)
    
    # Layer 2
    layer2_weights = tf.Variable(tf.truncated_normal(
        shape=(patch_size, patch_size, depth, depth), stddev=0.1))
    layer2_biases = tf.Variable(tf.ones(shape=(depth,)))
    
    conv2 = tf.nn.conv2d(hidden1, layer2_weights, strides=[1, 2, 2, 1], padding='SAME')
    hidden2 = tf.nn.relu(conv2 + layer2_biases)
    
    shape2 = hidden2.get_shape().as_list()
    hidden2_reshape = tf.reshape(hidden2, [-1, shape2[1] * shape2[2] * shape2[3]])

    # Layer 3
    layer3_weights = tf.Variable(tf.truncated_normal(
        shape=(image_size // 4 * image_size // 4 * depth, hidden_size), stddev=0.1))
    layer3_biases = tf.Variable(tf.ones(shape=(hidden_size,)))
    
    hidden3 = tf.nn.relu_layer(hidden2_reshape, layer3_weights, layer3_biases)
    
    # Layer 4
    layer4_weights = tf.Variable(tf.truncated_normal(
        shape=(hidden_size, num_labels), stddev=0.1))
    layer4_biases = tf.Variable(tf.ones(shape=(num_labels,)))
      
    logits = tf.nn.xw_plus_b(hidden3, layer4_weights, layer4_biases)
  
    # Output
    prediction = tf.nn.softmax(logits)

    # Training computation.
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    return X, y, prediction, loss, optimizer

In [6]:
def randomize(dataset, labels):
    permutation = np.random.permutation(labels.shape[0])
    shuffled_dataset = dataset[permutation, :, :, :]
    shuffled_labels = labels[permutation]
    return shuffled_dataset, shuffled_labels

def train(model_fn,
          num_epochs,
          batch_size,
          train_dataset=train_dataset,
          train_labels=train_labels,
          shuffle_epoch=True):
    
    epoch_size = train_labels.shape[0]
    epoch_size -= epoch_size % batch_size
    num_steps = epoch_size // batch_size
        
    with tf.Graph().as_default() as graph, \
        tf.Session(graph=graph) as session:

        X, y, prediction, loss_op, opt = model_fn()

        tf.global_variables_initializer().run()
        print('Initialized.\n')

        for epoch in range(num_epochs):
            print('Epoch {}\n'.format(epoch+1))
            
            if shuffle_epoch and epoch:
                print('Shuffling...\n')
                train_dataset, train_labels = randomize(train_dataset, train_labels)
            
            avg_loss = 0
            avg_acc = 0
            for step in range(num_steps):
                offset = step * batch_size
                batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
                batch_labels = train_labels[offset:(offset + batch_size), :]

                feed_dict = {X: batch_data, y: batch_labels}
                _, loss, predictions = session.run(
                    [opt, loss_op, prediction], feed_dict=feed_dict)
                
                avg_loss = (loss + step * avg_loss) / (step + 1)
                
                acc = accuracy(predictions, batch_labels)
                avg_acc = (acc + step * avg_acc) / (step + 1)

            print('Avarage loss: {:.3f}'.format(avg_loss))
            print('Avarage accuracy: {:.1f}%'.format(avg_acc))
 
            valid_pred = session.run(prediction, feed_dict={X: valid_dataset}) 
            valid_acc = accuracy(valid_pred, valid_labels)
            print('Validation accuracy: {:.1f}%\n'.format(valid_acc))

        test_pred = session.run(prediction, feed_dict={X: test_dataset})
        test_acc = accuracy(test_pred, test_labels)
        print('\nTest accuracy: {:.2f}%\n'.format(test_acc))

In [7]:
image_size = 28
num_channels = 1
num_labels = 10
patch_size = 5
depth = 16
hidden_size = 64

train(lambda: convnet_5(image_size, num_channels, num_labels, patch_size, depth, hidden_size),
      num_epochs=5,
      batch_size=16)

Initialized.

Epoch 1

Avarage loss: 0.449
Avarage accuracy: 86.3%
Validation accuracy: 88.8%

Epoch 2

Shuffling...

Avarage loss: 0.338
Avarage accuracy: 89.6%
Validation accuracy: 90.1%

Epoch 3

Shuffling...

Avarage loss: 0.305
Avarage accuracy: 90.6%
Validation accuracy: 90.4%

Epoch 4

Shuffling...

Avarage loss: 0.283
Avarage accuracy: 91.2%
Validation accuracy: 90.3%

Epoch 5

Shuffling...

Avarage loss: 0.267
Avarage accuracy: 91.7%
Validation accuracy: 90.8%


Test accuracy: 95.98%



---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [10]:
def convnet_5_pooling(image_size,
                      num_channels,
                      num_labels,
                      patch_size,
                      depth,
                      hidden_size):
    X = tf.placeholder(shape=(None, image_size, image_size, num_channels), dtype=tf.float32)
    y = tf.placeholder(shape=(None, num_labels), dtype=tf.float32)

    # Layer 1
    layer1_weights = tf.Variable(tf.truncated_normal(
        shape=(patch_size, patch_size, num_channels, depth), stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros(shape=(depth,)))
    
    conv1 = tf.nn.conv2d(X, layer1_weights, strides=[1, 1, 1, 1], padding='SAME')
    hidden1 = tf.nn.relu(conv1 + layer1_biases)
    pool1 = tf.nn.max_pool(hidden1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    # Layer 2
    layer2_weights = tf.Variable(tf.truncated_normal(
        shape=(patch_size, patch_size, depth, depth), stddev=0.1))
    layer2_biases = tf.Variable(tf.ones(shape=(depth,)))
    
    conv2 = tf.nn.conv2d(pool1, layer2_weights, strides=[1, 1, 1, 1], padding='SAME')
    hidden2 = tf.nn.relu(conv2 + layer2_biases)
    pool2 = tf.nn.max_pool(hidden2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    shape2 = pool2.get_shape().as_list()
    pool2_reshape = tf.reshape(pool2, [-1, shape2[1] * shape2[2] * shape2[3]])

    # Layer 3
    layer3_weights = tf.Variable(tf.truncated_normal(
        shape=(image_size // 4 * image_size // 4 * depth, hidden_size), stddev=0.1))
    layer3_biases = tf.Variable(tf.ones(shape=(hidden_size,)))
    
    hidden3 = tf.nn.relu_layer(pool2_reshape, layer3_weights, layer3_biases)
    
    # Layer 4
    layer4_weights = tf.Variable(tf.truncated_normal(
        shape=(hidden_size, num_labels), stddev=0.1))
    layer4_biases = tf.Variable(tf.ones(shape=(num_labels,)))
      
    logits = tf.nn.xw_plus_b(hidden3, layer4_weights, layer4_biases)
  
    # Output
    prediction = tf.nn.softmax(logits)

    # Training computation.
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    return X, y, prediction, loss, optimizer


train(lambda: convnet_5_pooling(image_size, num_channels, num_labels, patch_size, depth, hidden_size),
      num_epochs=5,
      batch_size=16)

Initialized.

Epoch 1

Avarage loss: 0.417
Avarage accuracy: 87.4%
Validation accuracy: 89.8%

Epoch 2

Shuffling...

Avarage loss: 0.315
Avarage accuracy: 90.3%
Validation accuracy: 90.5%

Epoch 3

Shuffling...

Avarage loss: 0.287
Avarage accuracy: 91.2%
Validation accuracy: 90.8%

Epoch 4

Shuffling...

Avarage loss: 0.268
Avarage accuracy: 91.7%
Validation accuracy: 91.3%

Epoch 5

Shuffling...

Avarage loss: 0.254
Avarage accuracy: 92.1%
Validation accuracy: 91.4%


Test accuracy: 96.42%



---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [13]:
def convnet_5_dropout(image_size,
                      num_channels,
                      num_labels,
                      patch_size,
                      depth,
                      hidden_size,
                      dropout_keep_prob=0.5,
                      use_dropout=True):
    X = tf.placeholder(shape=(None, image_size, image_size, num_channels), dtype=tf.float32)
    y = tf.placeholder(shape=(None, num_labels), dtype=tf.float32)

    # Layer 1
    layer1_weights = tf.Variable(tf.truncated_normal(
        shape=(patch_size, patch_size, num_channels, depth), stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros(shape=(depth,)))
    
    conv1 = tf.nn.conv2d(X, layer1_weights, strides=[1, 1, 1, 1], padding='SAME')
    hidden1 = tf.nn.relu(conv1 + layer1_biases)
    pool1 = tf.nn.max_pool(hidden1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    # Layer 2
    layer2_weights = tf.Variable(tf.truncated_normal(
        shape=(patch_size, patch_size, depth, depth), stddev=0.1))
    layer2_biases = tf.Variable(tf.ones(shape=(depth,)))
    
    conv2 = tf.nn.conv2d(pool1, layer2_weights, strides=[1, 1, 1, 1], padding='SAME')
    hidden2 = tf.nn.relu(conv2 + layer2_biases)
    pool2 = tf.nn.max_pool(hidden2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    shape2 = pool2.get_shape().as_list()
    pool2_reshape = tf.reshape(pool2, [-1, shape2[1] * shape2[2] * shape2[3]])

    # Layer 3
    layer3_weights = tf.Variable(tf.truncated_normal(
        shape=(image_size // 4 * image_size // 4 * depth, hidden_size), stddev=0.1))
    layer3_biases = tf.Variable(tf.ones(shape=(hidden_size,)))
    
    hidden3 = tf.nn.relu_layer(pool2_reshape, layer3_weights, layer3_biases)
    
    # Layer 4
    layer4_weights = tf.Variable(tf.truncated_normal(
        shape=(hidden_size, num_labels), stddev=0.1))
    layer4_biases = tf.Variable(tf.ones(shape=(num_labels,)))
    
    logits = tf.nn.xw_plus_b(hidden3, layer4_weights, layer4_biases)
  
    # Output
    prediction = tf.nn.softmax(logits)

    # Training computation.
    if use_dropout:
        hidden3 = tf.nn.dropout(hidden3, dropout_keep_prob)
        logits = tf.nn.xw_plus_b(hidden3, layer4_weights, layer4_biases)
        
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits))
    
    return X, y, prediction, loss

model_fn = lambda: convnet_5_dropout(image_size, num_channels, num_labels, patch_size, depth, hidden_size)


def model_opt_decay(model_fn,
              starter_learning_rate=0.05,
              decay_steps=10_000,
              decay_rate=0.98,
              staircase=True):    
    X, y, prediction, loss = model_fn()
    
    global_step = tf.Variable(0, trainable=False)
    learning_rate = tf.train.exponential_decay(
        starter_learning_rate, global_step, decay_steps, decay_rate, staircase=staircase)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    return X, y, prediction, loss, optimizer

train(lambda: model_opt_decay(model_fn),
      num_epochs=10,
      batch_size=16)

Initialized.

Epoch 1

Avarage loss: 0.575
Avarage accuracy: 85.8%
Validation accuracy: 88.9%

Epoch 2

Shuffling...

Avarage loss: 0.427
Avarage accuracy: 89.1%
Validation accuracy: 89.8%

Epoch 3

Shuffling...

Avarage loss: 0.396
Avarage accuracy: 90.0%
Validation accuracy: 90.2%

Epoch 4

Shuffling...

Avarage loss: 0.375
Avarage accuracy: 90.4%
Validation accuracy: 90.5%

Epoch 5

Shuffling...

Avarage loss: 0.364
Avarage accuracy: 90.7%
Validation accuracy: 90.4%

Epoch 6

Shuffling...

Avarage loss: 0.354
Avarage accuracy: 91.0%
Validation accuracy: 90.7%

Epoch 7

Shuffling...

Avarage loss: 0.346
Avarage accuracy: 91.2%
Validation accuracy: 91.0%

Epoch 8

Shuffling...

Avarage loss: 0.337
Avarage accuracy: 91.3%
Validation accuracy: 90.9%

Epoch 9

Shuffling...

Avarage loss: 0.332
Avarage accuracy: 91.5%
Validation accuracy: 91.3%

Epoch 10

Shuffling...

Avarage loss: 0.327
Avarage accuracy: 91.7%
Validation accuracy: 91.3%


Test accuracy: 96.20%



In [14]:
def model_opt_adam(model_fn, learning_rate=0.0001, clip_grads=1):
    X, y, prediction, loss = model_fn()

    opt_fn = tf.train.AdamOptimizer(learning_rate)
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), clip_grads)
    optimizer = opt_fn.apply_gradients(zip(grads, tvars))
    
    return X, y, prediction, loss, optimizer

train(lambda: model_opt_adam(model_fn),
      num_epochs=10,
      batch_size=16)

Initialized.

Epoch 1

Avarage loss: 0.875
Avarage accuracy: 80.0%
Validation accuracy: 85.2%

Epoch 2

Shuffling...

Avarage loss: 0.581
Avarage accuracy: 86.2%
Validation accuracy: 87.0%

Epoch 3

Shuffling...

Avarage loss: 0.521
Avarage accuracy: 87.5%
Validation accuracy: 88.0%

Epoch 4

Shuffling...

Avarage loss: 0.483
Avarage accuracy: 88.2%
Validation accuracy: 88.9%

Epoch 5

Shuffling...

Avarage loss: 0.456
Avarage accuracy: 88.8%
Validation accuracy: 89.1%

Epoch 6

Shuffling...

Avarage loss: 0.438
Avarage accuracy: 89.3%
Validation accuracy: 89.6%

Epoch 7

Shuffling...

Avarage loss: 0.426
Avarage accuracy: 89.6%
Validation accuracy: 89.8%

Epoch 8

Shuffling...

Avarage loss: 0.414
Avarage accuracy: 89.9%
Validation accuracy: 89.9%

Epoch 9

Shuffling...

Avarage loss: 0.403
Avarage accuracy: 90.1%
Validation accuracy: 90.0%

Epoch 10

Shuffling...

Avarage loss: 0.395
Avarage accuracy: 90.4%
Validation accuracy: 90.1%


Test accuracy: 95.66%

